# TASK 1 - Regresión Lineal

### Responda a cada una de las siguientes preguntas de forma clara y de lo más completamente posible.

1. Considera un modelo de regresión lineal con dos características, X₁ y X₂, y sus pesos correspondientes w₁ y
w₂. Si el modelo predice una salida y mediante la ecuación y = 2w₁X₁ + 3w₂X₂ + 1, ¿cuál es la interpretación
del coeficiente 3w₂ en el contexto del modelo?

R//

2. Explica el concepto de multicolinealidad en el contexto de la regresión lineal. ¿Cómo afecta la
multicolinealidad a la interpretación de los coeficientes de regresión individuales?

R//

# TASK 2 - Exploración de datos

### Importar librerías y Configurar Entorno

In [2]:
# Manipulación y visualización
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocesamiento
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.preprocessing import LabelEncoder

# Métrica de evaluación
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Modificar máximo de columnas
pd.set_option('display.max_columns', None)


### Importar el Dataset

In [11]:
data = pd.read_csv('dataset_phishing.csv')

## Exploración de datos

In [12]:
data.head()

,url,length_url,length_hostname,ip,nb_dots,nb_hyphens,nb_at,nb_qm,nb_and,nb_or,nb_eq,nb_underscore,nb_tilde,nb_percent,nb_slash,nb_star,nb_colon,nb_comma,nb_semicolumn,nb_dollar,nb_space,nb_www,nb_com,nb_dslash,http_in_path,https_token,ratio_digits_url,ratio_digits_host,punycode,port,tld_in_path,tld_in_subdomain,abnormal_subdomain,nb_subdomains,prefix_suffix,random_domain,shortening_service,path_extension,nb_redirection,nb_external_redirection,length_words_raw,char_repeat,shortest_words_raw,shortest_word_host,shortest_word_path,longest_words_raw,longest_word_host,longest_word_path,avg_words_raw,avg_word_host,avg_word_path,phish_hints,domain_in_brand,brand_in_subdomain,brand_in_path,suspecious_tld,statistical_report,nb_hyperlinks,ratio_intHyperlinks,ratio_extHyperlinks,ratio_nullHyperlinks,nb_extCSS,ratio_intRedirection,ratio_extRedirection,ratio_intErrors,ratio_extErrors,login_form,external_favicon,links_in_tags,submit_email,ratio_intMedia,ratio_extMedia,sfh,iframe,popup_window,safe_anchor,onmouseover,right_clic,empty_title,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status
0,http://www.crestonwood.com/router.php,37,19,0,3,0,0,0,0,0,0,0,0,0,3,0,1,0,0,0,0,1,0,0,0,1,0.000000,0.0,0,0,0,0,0,3,0,0,0,0,0,0,4,4,3,3,3,11,11,6,5.750000,7.0,4.500000,0,0,0,0,0,0,17,0.529412,0.470588,0,0,0,0.875000,0,0.500000,0,0,80.000000,0,100.000000,0.000000,0,0,0,0.0,0,0,0,0,1,0,45,-1,0,1,1,4,legitimate
1,http://shadetreetechnology.com/V4/validation/a...,77,23,1,1,0,0,0,0,0,0,0,0,0,5,0,1,0,0,0,0,0,0,0,0,1,0.220779,0.0,0,0,0,0,0,1,0,0,0,0,1,0,4,4,2,19,2,32,19,32,15.750000,19.0,14.666667,0,0,0,0,0,0,30,0.966667,0.033333,0,0,0,0.000000,0,0.000000,0,0,100.000000,0,80.000000,20.000000,0,0,0,100.0,0,0,0,1,0,0,77,5767,0,0,1,2,phishing
2,https://support-appleld.com.secureupdate.duila...,126,50,1,4,1,0,1,2,0,3,2,0,0,5,0,1,0,0,0,0,0,1,0,0,0,0.150794,0.0,0,0,0,1,0,3,1,0,0,0,1,0,12,2,2,3,2,17,13,17,8.250000,8.4,8.142857,0,0,0,0,0,0,4,1.000000,0.000000,0,0,0,0.000000,0,0.000000,0,0,100.000000,0,0.000000,0.000000,0,0,0,100.0,0,0,0,1,0,0,14,4004,5828815,0,1,0,phishing
3,http://rgipt.ac.in,18,11,0,2,0,0,0,0,0,0,0,0,0,2,0,1,0,0,0,0,0,0,0,0,1,0.000000,0.0,0,0,0,0,0,2,0,0,0,0,1,0,1,0,5,5,0,5,5,0,5.000000,5.0,0.000000,0,0,0,0,0,0,149,0.973154,0.026846,0,0,0,0.250000,0,0.250000,0,0,100.000000,0,96.428571,3.571429,0,0,0,62.5,0,0,0,1,0,0,62,-1,107721,0,0,3,legitimate
4,http://www.iracing.com/tracks/gateway-motorspo...,55,15,0,2,2,0,0,0,0,0,0,0,0,5,0,1,0,0,0,0,1,0,0,0,1,0.000000,0.0,0,0,0,0,0,2,0,0,0,0,1,0,6,3,3,3,4,11,7,11,6.333333,5.0,7.000000,0,0,0,0,0,0,102,0.470588,0.529412,0,0,0,0.537037,0,0.018519,1,0,76.470588,0,0.000000,100.000000,0,0,0,0.0,0,0,0,0,1,0,224,8175,8725,0,0,6,legitimate


In [11]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11430 entries, 0 to 11429
Data columns (total 89 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   url                         11430 non-null  object 
 1   length_url                  11430 non-null  int64  
 2   length_hostname             11430 non-null  int64  
 3   ip                          11430 non-null  int64  
 4   nb_dots                     11430 non-null  int64  
 5   nb_hyphens                  11430 non-null  int64  
 6   nb_at                       11430 non-null  int64  
 7   nb_qm                       11430 non-null  int64  
 8   nb_and                      11430 non-null  int64  
 9   nb_or                       11430 non-null  int64  
 10  nb_eq                       11430 non-null  int64  
 11  nb_underscore               11430 non-null  int64  
 12  nb_tilde                    11430 non-null  int64  
 13  nb_percent                  114

In [13]:
data.describe()

,length_url,length_hostname,ip,nb_dots,nb_hyphens,nb_at,nb_qm,nb_and,nb_or,nb_eq,nb_underscore,nb_tilde,nb_percent,nb_slash,nb_star,nb_colon,nb_comma,nb_semicolumn,nb_dollar,nb_space,nb_www,nb_com,nb_dslash,http_in_path,https_token,ratio_digits_url,ratio_digits_host,punycode,port,tld_in_path,tld_in_subdomain,abnormal_subdomain,nb_subdomains,prefix_suffix,random_domain,shortening_service,path_extension,nb_redirection,nb_external_redirection,length_words_raw,char_repeat,shortest_words_raw,shortest_word_host,shortest_word_path,longest_words_raw,longest_word_host,longest_word_path,avg_words_raw,avg_word_host,avg_word_path,phish_hints,domain_in_brand,brand_in_subdomain,brand_in_path,suspecious_tld,statistical_report,nb_hyperlinks,ratio_intHyperlinks,ratio_extHyperlinks,ratio_nullHyperlinks,nb_extCSS,ratio_intRedirection,ratio_extRedirection,ratio_intErrors,ratio_extErrors,login_form,external_favicon,links_in_tags,submit_email,ratio_intMedia,ratio_extMedia,sfh,iframe,popup_window,safe_anchor,onmouseover,right_clic,empty_title,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank
count,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.0,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.0,11430.000000,11430.0,11430.000000,11430.0,11430.000000,11430.000000,11430.000000,11430.000000,11430.0,11430.000000,11430.000000,11430.0,11430.000000,11430.000000,11430.000000,11430.000000,11430.00000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,1.143000e+04,11430.000000,11430.000000,11430.000000
mean,61.126684,21.090289,0.150569,2.480752,0.997550,0.022222,0.141207,0.162292,0.0,0.293176,0.322660,0.006649,0.123097,4.289589,0.000700,1.027909,0.004024,0.062292,0.001925,0.034821,0.448469,0.127997,0.006562,0.016710,0.610936,0.053137,0.025024,0.000350,0.002362,0.065617,0.050131,0.021610,2.231671,0.202450,0.083290,0.123447,0.000175,0.498250,0.003150,6.232808,2.927472,3.127297,5.019773,2.398950,15.393876,10.467979,10.561505,7.258882,7.678075,5.092425,0.327734,0.104199,0.004112,0.004899,0.017935,0.059755,87.189764,0.602457,0.276720,0.0,0.784864,0.0,0.158926,0.0,0.062469,0.063605,0.442170,51.978211,0.0,42.870444,23.236293,0.0,0.001312,0.006037,37.063922,0.001137,0.00140,0.124759,0.775853,0.439545,0.072878,492.532196,4062.543745,8.567566e+05,0.020122,0.533946,3.185739
std,55.297318,10.777171,0.357644,1.369686,2.087087,0.155500,0.364456,0.821337,0.0,0.998317,1.093336,0.081274,1.466450,1.882251,0.026448,0.240325,0.103240,0.598190,0.077111,0.375576,0.501912,0.379008,0.080742,0.169358,0.487559,0.089363,0.093422,0.018705,0.048547,0.247622,0.218225,0.145412,0.637069,0.401843,0.276332,0.328964,0.013227,0.691907,0.056035,5.572355,4.768936,2.211571,3.941580,2.997809,22.083644,4.932015,23.077883,4.145827,3.578435,7.147050,0.842600,0.305533,0.063996,0.069827,0.132722,0.331266,166.758254,0.376474,0.319958,0.0,2.758802,0.0,0.266437,0.0,0.156209,0.244058,0.496666,41.523144,0.0,46.249897,38.386577,0.0,0.036204,0.077465,39.073385,0.033707,0.03739,0.330460,0.417038,0.496353,0.259948,814.769415,3107.784600,1.995606e+06,0.140425,0.498868,2.536955
min,12.000000,4.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,1.000000,0.000000,0.000000

### Verificar si hay datos duplicados

In [13]:
print(not data["url"].is_unique)

True


No hay urls duplicadas

### Excluir Columnas innecesarias

Se excluirán las siguientes columnas:
- ```URL```: Ya hay columnas describiendo el contenido de las urls, por lo que esta columna es innecesaria
-  Por el momento se guardará la columna ```Status``` en ```Y``` para la métrica de desempeño.

In [17]:
X = data.iloc[:,1:-1]
X = X.values
Y = data.iloc[:,-1].values

print(X)
print(Y)

[[ 37.  19.   0. ...   1.   1.   4.]
 [ 77.  23.   1. ...   0.   1.   2.]
 [126.  50.   1. ...   0.   1.   0.]
 ...
 [105.  16.   1. ...   0.   1.  10.]
 [ 38.  30.   0. ...   0.   0.   4.]
 [477.  14.   1. ...   1.   1.   0.]]
['legitimate' 'phishing' 'phishing' ... 'legitimate' 'legitimate'
 'phishing']


### Revisión de datos faltantes

In [20]:
pd.DataFrame(X).isnull().sum()


0     0
1     0
2     0
3     0
4     0
     ..
82    0
83    0
84    0
85    0
86    0
Length: 87, dtype: int64

No se encontraron datos faltantes

## División del dataset en conjuntos de entrenamiento y prueba

Se dividirán los datos de modo de que el 80% sea destinado para el **entreno** y el 20% para el **testing**.

Esto para evitar sesgos en la evaluación y que los datos de prueba no afecten el entrenamiento.

In [24]:
# Dividir el dataset en conjunto de entrenamiento (80%) y prueba (20%)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=33)

print("Los datos se dividieron de la siguiente manera:")
print("- Entrenamiento:", X_train.shape)
print("- Prueba:", X_test.shape)


Los datos se dividieron de la siguiente manera:
- Entrenamiento: (9144, 87)
- Prueba: (2286, 87)


Ahora con los datos divididos correctamente, el siguiente paso es Escalar y Codificar variables

## Codificar Variables Categóricas

La única variable categórica que es necesario codificar es la de ```status```, la cuál puede tener un valor de ```phishing``` o ```legitimate```

In [25]:
print("Y_train antes de la codificación:", Y_train[:5])
label_encoder = LabelEncoder()
Y_train = label_encoder.fit_transform(Y_train)
Y_test = label_encoder.transform(Y_test)

print("Y_train después de la codificación:", Y_train[:5])
print("Clases codificadas:", label_encoder.classes_)

Y_train antes de la codificación: ['phishing' 'legitimate' 'legitimate' 'legitimate' 'legitimate']
Y_train después de la codificación: [1 0 0 0 0]
Clases codificadas: ['legitimate' 'phishing']


Ahora que las categorías **legitimate** y **phishing** se codificaron a **1** y **0**, se continuará con el escalado de variables.

### Escalado de Variables
Este escalado se aplica después de la división de datos para evitar que la información de los datos de prueba contamine el entrenamiento.
- Se escala para asegurar que todas las variables tengan la misma influencia en el futuro.

Primero se hará una inspección de los valores para determinar la mejor manera de escalar los datos de ser necesario.

In [26]:
X_train_df = pd.DataFrame(X_train, columns=data.columns[1:-1])
X_test_df = pd.DataFrame(X_test, columns=data.columns[1:-1])
X_train_df.describe()


,length_url,length_hostname,ip,nb_dots,nb_hyphens,nb_at,nb_qm,nb_and,nb_or,nb_eq,nb_underscore,nb_tilde,nb_percent,nb_slash,nb_star,nb_colon,nb_comma,nb_semicolumn,nb_dollar,nb_space,nb_www,nb_com,nb_dslash,http_in_path,https_token,ratio_digits_url,ratio_digits_host,punycode,port,tld_in_path,tld_in_subdomain,abnormal_subdomain,nb_subdomains,prefix_suffix,random_domain,shortening_service,path_extension,nb_redirection,nb_external_redirection,length_words_raw,char_repeat,shortest_words_raw,shortest_word_host,shortest_word_path,longest_words_raw,longest_word_host,longest_word_path,avg_words_raw,avg_word_host,avg_word_path,phish_hints,domain_in_brand,brand_in_subdomain,brand_in_path,suspecious_tld,statistical_report,nb_hyperlinks,ratio_intHyperlinks,ratio_extHyperlinks,ratio_nullHyperlinks,nb_extCSS,ratio_intRedirection,ratio_extRedirection,ratio_intErrors,ratio_extErrors,login_form,external_favicon,links_in_tags,submit_email,ratio_intMedia,ratio_extMedia,sfh,iframe,popup_window,safe_anchor,onmouseover,right_clic,empty_title,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank
count,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9144.0,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9144.0,9144.000000,9144.0,9144.000000,9144.0,9144.000000,9144.000000,9144.000000,9144.000000,9144.0,9144.000000,9144.000000,9144.0,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9144.000000,9.144000e+03,9144.000000,9144.000000,9144.000000
mean,60.846238,21.063320,0.147310,2.483596,0.993438,0.022528,0.138561,0.159558,0.0,0.287183,0.319444,0.006343,0.118329,4.284339,0.000766,1.028434,0.003937,0.062227,0.001640,0.036199,0.451553,0.128828,0.006671,0.016951,0.610127,0.052178,0.024783,0.000328,0.002515,0.065617,0.048338,0.020013,2.233377,0.204834,0.083880,0.124453,0.000109,0.499891,0.003062,6.201990,2.906715,3.140201,5.008968,2.395232,15.372266,10.482612,10.500984,7.266703,7.679780,5.066113,0.324584,0.102800,0.003500,0.005031,0.017060,0.057852,87.780402,0.606515,0.275156,0.0,0.779418,0.0,0.159726,0.0,0.062165,0.062445,0.445757,52.351839,0.0,43.045988,23.423828,0.0,0.001312,0.006343,37.363254,0.001312,0.001640,0.122922,0.776575,0.442367,0.071085,491.915245,4073.397638,8.445066e+05,0.018810,0.526465,3.188867
std,55.024461,10.608823,0.354434,1.388597,2.069333,0.158385,0.362499,0.825083,0.0,0.997309,1.108166,0.079394,1.523593,1.899956,0.027659,0.243162,0.100234,0.569331,0.058206,0.397826,0.501833,0.382294,0.081408,0.173861,0.487748,0.088266,0.091948,0.018111,0.050092,0.247625,0.214491,0.140053,0.636831,0.403602,0.277223,0.330116,0.010458,0.696549,0.055255,5.599096,4.311270,2.256112,3.932066,3.026798,21.236121,4.963139,22.263513,4.127055,3.573905,7.135332,0.839566,0.303714,0.059057,0.070752,0.129503,0.326790,168.075199,0.373855,0.317102,0.0,2.728281,0.0,0.266697,0.0,0.155045,0.241976,0.497076,41.326801,0.0,46.240269,38.472794,0.0,0.036204,0.079394,39.010245,0.036204,0.040471,0.328366,0.416564,0.496694,0.256981,833.824922,3117.262222,1.980062e+06,0.135861,0.499326,2.529165
min,12.000000,4.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0

### Observaciones de los valores
- ```length_url``` tiene un valor máximo de 1641, lejano a la media de 60 caracteres por lo que puede tratarse de un valor atípico.
- Hay variables binarias que no se necesitan escalar, las cuales son: ```tdl_in_path```, ```http_in_path```, ```is_active```, ```has_error```, ```is_authenticated```

In [29]:
X_train_df['length_url'].nlargest(n=5)

6182    1641.0
6800    1386.0
7848    1386.0
6668     907.0
425      795.0
Name: length_url, dtype: float64

Se puede notar que el dato mayor de ```length_url``` no es un valor atípico, ya que hay varias urls con un largo cercano, por lo que se descarta esa idea.

In [31]:
sc_X = StandardScaler()
columns = ['length_url', 'length_hostname', 'length_words_raw', 'char_repeat', 'longest_word_path',
'avg_words_raw', 'domain_registration_length', 'domain_age', 'web_traffic', 'page_rank']
X_train_df[columns] = sc_X.fit_transform(X_train_df[columns])
X_test_df[columns] = sc_X.fit_transform(X_test_df[columns])

print(X_train_df[columns].mean())
print(X_test_df[columns].mean())



length_url                   -2.719706e-18
length_hostname               1.981500e-17
length_words_raw             -3.108236e-17
char_repeat                  -3.360780e-17
longest_word_path             1.787236e-17
avg_words_raw                -9.324708e-18
domain_registration_length    3.885295e-18
domain_age                    1.340427e-17
web_traffic                  -3.496765e-17
page_rank                    -1.486125e-17
dtype: float64
length_url                   -7.770590e-18
length_hostname              -6.216472e-18
length_words_raw              6.216472e-18
char_repeat                   6.216472e-18
longest_word_path            -2.175765e-17
avg_words_raw                -4.662354e-18
domain_registration_length    1.554118e-18
domain_age                    2.175765e-17
web_traffic                  -3.108236e-18
page_rank                    -1.554118e-17
dtype: float64


In [32]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train, Y_train)

RandomForestClassifier()

In [33]:
from sklearn.metrics import classification_report
Y_pred = model.predict(X_test)
print(accuracy_score(Y_test, Y_pred))


0.9698162729658792


# Por el momento:
El dataframe está repartido en las siguientes variables:
- ```X_train``` : DF de entrenamiento
- ```X_test```: DF de prueba
- ```Y_test```: Status de url para entrenamiento
- ```X_test```: Status de url para pruebas

# TASK 2.1 - Regresión Logística 


#### IMPLEMENTACIÓN SIN LIBRERÍAS 

In [44]:
class LogisticRegression:
    def __init__(self, learning_rate=0.01, num_iterations=1000):
        self.learning_rate = learning_rate
        self.num_iterations = num_iterations
        self.weights = None
        self.bias = None
        


#### IMPLEMENTACIÓN CON LIBRERÍAS 

In [37]:
from sklearn.linear_model import LogisticRegression
sklearn_clasificaciones = LogisticRegression()
sklearn_clasificaciones.fit(X_train, Y_train)

c:\Users\rebe1\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
sklearnmodel_predictions = model.predict(X_test)

# TASK 2.2 - K-Nearest Neighbors

### Implementación sin librerías

In [23]:
# Función para calcular la distancia eucladiana entre dos puntos
def distancia_eucladiana(p1,p2):
    distancia = 0
    for i in range(len(p1)):
        distancia += (p1[i]-p2[i]) ** 2
    return distancia ** 0.5

In [24]:
# Función para obtener los k vecinos más cercanos de un punto de prueba
def obtener_k_vecinos(X_train, Y_train, punto_prueba, k):
    distancias = []
    for i in range(len(X_train)):
        distancia = distancia_eucladiana(punto_prueba, X_train[i])
        distancias.append((distancia, Y_train[i]))
    
    distancias_sorted = sorted(distancias, key = lambda x:x[0])
    k_vecinos = distancias_sorted[:k]

    return k_vecinos

In [26]:
# Función para clasificar un punto de prueba basado en los k vecinos más cercanos
def clasificar_punto(k_vecinos):
    contador = {}
    for vecino in k_vecinos:
        clase = vecino[1]
        if clase in contador:
            contador[clase] += 1
        else:
            contador[clase] = 1

    clase_mayoritaria = None
    max_frecuencia = 0
    for clase, frecuencia in contador.items():
        if frecuencia > max_frecuencia:
            max_frecuencia = frecuencia
            clase_mayoritaria = clase
            
    return clase_mayoritaria

In [27]:
# Función principal del algoritmo KNN
# Clasifica cada punto del conjunto de prueba basándose en los datos de entrenamiento y el valor de k
def knn(X_train, Y_train, X_test, k):
    predicciones = []
    for punto_prueba in X_test:
        k_vecinos = obtener_k_vecinos(X_train, Y_train, punto_prueba, k)
        clase = clasificar_punto(k_vecinos)
        predicciones.append(clase)

    return predicciones

In [28]:
# Función para calcular la precisión del modelo
def calcular_accuracy(y_real, y_pred):
    aciertos = 0
    for i in range(len(y_real)):
        if y_real[i] == y_pred[i]:
            aciertos += 1
    return aciertos / len(y_real)

In [ ]:
# Calcula el accuracy del modelo con base al número de k establecido
k = 3
y_pred = knn(X_train, Y_train, X_test, k)
accuracy = calcular_accuracy(Y_test, y_pred)
print(f"Accuracy del modelo: {accuracy * 100:.2f}%")

Accuracy del modelo: 81.54%


### Implementación con librerías

In [67]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

k = 3 
knn_model = KNeighborsClassifier(n_neighbors=k)

# Entrenar el modelo
knn_model.fit(X_train, Y_train)

# Hacer predicciones en el conjunto de prueba
y_pred = knn_model.predict(X_test)

accuracy = accuracy_score(Y_test, y_pred)
print(f"Accuracy del modelo: {accuracy * 100:.2f}%")

Accuracy del modelo: 83.77%


## Grupos encontrados
